<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/animate_satellite_data_hurr_track.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install modules

In [ ]:
!pip install tropycal
!pip install copernicusmarine
!pip install cartopy

In [ ]:
import copernicusmarine
from tropycal import tracks
import xarray as xr
import numpy as np
import pandas as pd

from PIL import Image
import glob as glob

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
#import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

#%matplotlib inline

In [ ]:
 def get_cmems_data(dsetID, vNames, lon_start, lon_end, lat_start, lat_end, time_start, time_end):

  data_request = {"dataset_id" : dsetID,
    "longitude" : [lon_start, lon_end],
    "latitude" : [lat_start, lat_end],
    "time" : [time_start, time_end],
    "variables" : vNames}

  cms_data =copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    variables = data_request["variables"])

  return cms_data

In [ ]:
# Look into this alternative:
# https://stackoverflow.com/questions/78639092/how-to-animation-with-xarray-dataset-and-cartopy-projection-with-a-colorbar

def plot_l4_hurrTrack(dataArr, vName, hurr, xMin, xMax, yMin, yMax, vMin, vMax, cMap, width=8, height=6, DPI=120, track_color='k'):

  nTime=len(dataArr.time)
  for iTime in range(nTime):

    fig = plt.figure(figsize=(width,height), dpi=DPI)

    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_extent([xMin, xMax, yMin, yMax], crs=ccrs.PlateCarree())

    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

    date1 = dataArr.time[iTime].values
    yyyymmdd = date1.astype('str').split('T')[0]
    hh = date1.astype('str').split('T')[1].split(':')[0]
    title_str = yyyymmdd+'T'+hh

    dataArr.isel(time=iTime).plot(ax=ax, vmin=vMin, vmax=vMax, cmap=cMap,
                                  cbar_kwargs={'pad':0.01, 'shrink':0.75})
    # Overlay hurricane when it went by
    if date1 >= hurr.time[0].values and date1 <= hurr.time[-1].values:
      ax.scatter(hurr.sel(time=slice(hurr.time[0], date1)).lon,\
            hurr.sel(time=slice(hurr.time[0], date1)).lat,\
            s=hurr.sel(time=slice(hurr.time[0], date1)).vmax/5., marker='x', c=track_color, transform=ccrs.PlateCarree())

    ax.set_title(title_str, fontsize=14)
    figName = 'l4_'+vName+'_'+title_str+'.png'
    plt.savefig(figName)
    plt.close('all')

In [ ]:
# make a gif animation from png files
def make_gif_from_pngs(data_path, fPref, png_files, fSuff='.gif', DUR=1000, LOOP=0):
  images = []
  for filename in png_files:
    im = Image.open(filename)
    images.append(im)

  # save as a gif
  fOut = data_path+fPref + fSuff
  images[0].save(fOut,save_all=True,
               append_images=images[0:],
               optimize=False,
               duration=DUR, # Duration in milliseconds
               loop=LOOP) # infinite loop
  print('\nSaved:\t{}\n'.format(fOut))

## User Inputs

In [ ]:
# Basin and year
myBasin = 'north_atlantic'
year = 2024
hurr_name = "Milton"
time_delta = 5 # days before/after storm
dLon, dLat = [5, 5] # plot extra data outside track bounds (in degrees)

### Dataset specific info (may change with dataset, you need to fill it in _carefully_ by looking up [CMEMS](https://data.marine.copernicus.eu/products))


In [ ]:
# AVISO L4 SSH
def aviso_l4_ssh(year):

  if year < 2022:
    dsetID = "c3s_obs-sl_glo_phy-ssh_my_twosat-l4-duacs-0.25deg_P1D"
    vNames = ["adt", "sla", "err_sla", "ugos", "vgos"]
  else:
    dsetID = "cmems_obs-sl_glo_phy-ssh_nrt_allsat-l4-duacs-0.25deg_P1D"
    vNames = ["adt", "sla", "err_sla", "ugos", "vgos"]

  return dsetID, vNames

# CMEMS dataset IDs
dsetID_sst = 'METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2' # OSTIA SST
vNames_sst = ['analysed_sst']

dsetID_sss='cmems_obs-mob_glo_phy-sss_nrt_multi_P1D' # CNR SSS NRT product
vNames_sss=['sos', 'dos']

## Download hurricane track data

In [ ]:
basin = tracks.TrackDataset(basin=myBasin, source='hurdat', include_btk=True, interpolate_data=True)
hurr=basin.get_storm((hurr_name, year))
track_file = str(year)+'_'+hurr_name+'.nc'
hurr.to_xarray().to_netcdf(track_file)
print("\nSaved track info for {}, {} to:\n{}".format(hurr_name, year, track_file))

In [ ]:
# Hurricane formation and dissipation dates (yyyymmdd)
print("Hurricane: {},\t year: {}".format(hurr_name, year))
print("Formed on: {},\t dissipated on: {}".
format(hurr.time[0].strftime('%Y-%m-%d'), hurr.time[-1].strftime('%Y-%m-%d')))

t0 = hurr.time[0] - pd.Timedelta(days=time_delta)
t1 = hurr.time[-1] + pd.Timedelta(days=time_delta)

print("\nDownloading satellite data between following dates:")
print(t0.strftime('%Y-%m-%d'), "to", t1.strftime('%Y-%m-%d'))

## Get daily datasets of SSH, SST, SSS (satellite data products)

In [ ]:
# set credentials
copernicusmarine.login(username="sakella", password="HbFPyP9M")

### Data from AVISO (SSH, SLA, error in SLA, Geostrophic currents)

In [ ]:
l4_ssh = get_cmems_data(*aviso_l4_ssh(year),
                        hurr.lon.min()-dLon, hurr.lon.max()+dLon,
                        hurr.lat.min()-dLat, hurr.lat.max()+dLat,
                        t0.strftime('%Y-%m-%d'), t1.strftime('%Y-%m-%d'))

# add geostrophic currents to the dataset- eases our life!
l4_ssh['surf_curr'] = xr.DataArray(np.sqrt(l4_ssh.ugos**2 + l4_ssh.vgos**2), coords=l4_ssh.ugos.coords, dims=l4_ssh.ugos.dims)

## Data from OSTIA: SST

In [ ]:
l4_sst = get_cmems_data(dsetID_sst, vNames_sst,
                        hurr.lon.min()-dLon, hurr.lon.max()+dLon,
                        hurr.lat.min()-dLat, hurr.lat.max()+dLat,
                        t0.strftime('%Y-%m-%d'), t1.strftime('%Y-%m-%d'))

## Data from CNR (merged SMAP+SMOS): SSS

In [ ]:
l4_sss = get_cmems_data(dsetID_sss, vNames_sss,
                        hurr.lon.min()-dLon, hurr.lon.max()+dLon,
                        hurr.lat.min()-dLat, hurr.lat.max()+dLat,
                        t0.strftime('%Y-%m-%d'), t1.strftime('%Y-%m-%d'))

### Plot AVISO SLA and geostrophic currents

In [ ]:
vName = 'sla'
vMin, vMax, cMap = [-0.5, 0.5, 'RdBu_r']
xMin, xMax= [hurr.lon.min()-dLon, hurr.lon.max()+dLon]
yMin, yMax= [hurr.lat.min()-dLat, hurr.lat.max()+dLat]

plot_l4_hurrTrack(l4_ssh[vName], vName, xr.open_dataset(track_file), xMin, xMax, yMin, yMax, vMin, vMax, cMap)

# make a gif animation and delete png files
png_files_path, png_fPref = ['/content/',
                             hurr_name+'_'+str(year)+'_'+'l4_'+vName]
fNames = sorted( glob.glob( png_files_path + 'l4_'+vName +'*.png'))
#print(fNames)
make_gif_from_pngs(png_files_path, png_fPref, fNames)
!rm -f /content/l4_sla*.png
# ---

vName = 'surf_curr'
vMin, vMax, cMap = [0, 1, 'Blues']
xMin, xMax= [hurr.lon.min()-dLon, hurr.lon.max()+dLon]
yMin, yMax= [hurr.lat.min()-dLat, hurr.lat.max()+dLat]

plot_l4_hurrTrack(l4_ssh[vName], vName, xr.open_dataset(track_file), xMin, xMax, yMin, yMax, vMin, vMax, cMap)

# make a gif animation and delete png files
png_files_path, png_fPref = ['/content/',
                             hurr_name+'_'+str(year)+'_'+'l4_'+vName]
fNames = sorted( glob.glob( png_files_path + 'l4_'+vName+'*.png'))
#print(fNames)
make_gif_from_pngs(png_files_path, png_fPref, fNames)
!rm -f /content/l4_surf_curr*.png

## Plot OSTIA SST

In [ ]:
vName = 'analysed_sst'
vMin, vMax, cMap = [298., 308., 'gist_ncar']
xMin, xMax= [hurr.lon.min()-dLon, hurr.lon.max()+dLon]
yMin, yMax= [hurr.lat.min()-dLat, hurr.lat.max()+dLat]

plot_l4_hurrTrack(l4_sst[vName], vName, xr.open_dataset(track_file), xMin, xMax, yMin, yMax, vMin, vMax, cMap)

# make a gif animation and delete png files
png_files_path, png_fPref = ['/content/',
                             hurr_name+'_'+str(year)+'_'+'l4_'+vName]
fNames = sorted( glob.glob( png_files_path + 'l4_'+vName+'*.png'))
#print(fNames)
make_gif_from_pngs(png_files_path, png_fPref, fNames)
!rm -f /content/l4_analysed_sst*.png

## Plot CNR SSS and surface density

In [ ]:
vName = 'sos'
vMin, vMax, cMap = [33., 38., 'twilight_shifted']
xMin, xMax= [hurr.lon.min()-dLon, hurr.lon.max()+dLon]
yMin, yMax= [hurr.lat.min()-dLat, hurr.lat.max()+dLat]

plot_l4_hurrTrack(l4_sss[vName], vName, xr.open_dataset(track_file), xMin, xMax, yMin, yMax, vMin, vMax, cMap)

# make a gif animation and delete png files
png_files_path, png_fPref = ['/content/',
                             hurr_name+'_'+str(year)+'_'+'l4_'+vName]
fNames = sorted( glob.glob( png_files_path + 'l4_'+vName+'*.png'))
#print(fNames)
make_gif_from_pngs(png_files_path, png_fPref, fNames)
!rm -f /content/l4_sos*.png
# --

vName = 'dos'
vMin, vMax, cMap = [1020., 1024., 'twilight_shifted']
xMin, xMax= [hurr.lon.min()-dLon, hurr.lon.max()+dLon]
yMin, yMax= [hurr.lat.min()-dLat, hurr.lat.max()+dLat]

plot_l4_hurrTrack(l4_sss[vName], vName, xr.open_dataset(track_file), xMin, xMax, yMin, yMax, vMin, vMax, cMap)

# make a gif animation and delete png files
png_files_path, png_fPref = ['/content/',
                             hurr_name+'_'+str(year)+'_'+'l4_'+vName]
fNames = sorted( glob.glob( png_files_path + 'l4_'+vName+'*.png'))
#print(fNames)
make_gif_from_pngs(png_files_path, png_fPref, fNames)
!rm -f /content/l4_dos*.png